# Exporting data from BigQuery to Google Cloud Storage

In this notebook, we export BigQuery data to GCS so that we can reuse our Keras model that was developed on CSV data.

In [1]:
# Run the chown command to change the ownership of the repository
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
# Install the Google Cloud BigQuery library
%pip install google-cloud-bigquery==1.25.0

     |████████████████████████████████| 169 kB 8.7 MB/s            
     |████████████████████████████████| 152 kB 62.0 MB/s            
     |████████████████████████████████| 93 kB 2.6 MB/s             
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.3.3
    Uninstalling google-auth-2.3.3:
      Successfully uninstalled google-auth-2.3.3
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.3.2
    Uninstalling google-api-core-2.3.2:
      Successfully uninstalled google-api-core-2.3.2
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 2.1.0
    Uninstalling google-resumable-media-2.1.0:
      Successfully uninstalled google-resumable-media-2.1.0
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 2.2.1
    Uninstalling google-cloud-core-2.2.1:
      Successfully uninstalled google-cloud-core-2.2.1
  Attempting uninstall:

Please ignore any incompatibility warnings and errors.
**Restart** the kernel to use updated packages. (On the Notebook menu, select Kernel > Restart Kernel > Restart).


In [3]:
# Importing necessary tensorflow library and printing the TF version.
import tensorflow as tf

print("Tensorflow version: ",tf.__version__)


Tensorflow version:  2.6.2


In [4]:
# The OS module in python provides functions for interacting with the operating system.
import os

from google.cloud import bigquery

Change the following cell as necessary:

In [5]:
# Change with your own bucket and project below:
BUCKET =  "qwiklabs-gcp-04-26c8364f1a75"
PROJECT = "qwiklabs-gcp-04-26c8364f1a75"

OUTDIR = "gs://{bucket}/taxifare/data".format(bucket=BUCKET)

os.environ['BUCKET'] = BUCKET
os.environ['OUTDIR'] = OUTDIR
os.environ['PROJECT'] = PROJECT

## Create BigQuery tables

If you haven not already created a BigQuery dataset for our data, run the following cell:

In [6]:
# Initialize a BigQuery client.
bq = bigquery.Client(project = PROJECT)
dataset = bigquery.Dataset(bq.dataset("taxifare"))

# Create a new dataset with the `create_dataset()` method.
try:
    bq.create_dataset(dataset)
    print("Dataset created")
except:
    print("Dataset already exists")

Dataset already exists


Let's create a table with 1 million examples.

Note that the order of columns is exactly what was in our CSV files.

In [7]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_training_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 1
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Executing query with job ID: 226cff9e-883b-4808-a841-f5dbcb8819ae
Query executing: 0.35s


ERROR:
 404 Not found: Dataset qwiklabs-gcp-04-26c8364f1a75:taxifare was not found in location US

(job ID: 226cff9e-883b-4808-a841-f5dbcb8819ae)

                        -----Query Job SQL Follows-----                         

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:CREATE OR REPLACE TABLE taxifare.feateng_training_data AS
   2:
   3:SELECT
   4:    (tolls_amount + fare_amount) AS fare_amount,
   5:    pickup_datetime,
   6:    pickup_longitude AS pickuplon,
   7:    pickup_latitude AS pickuplat,
   8:    dropoff_longitude AS dropofflon,
   9:    dropoff_latitude AS dropofflat,
  10:    passenger_count*1.0 AS passengers,
  11:    'unused' AS key
  12:FROM `nyc-tlc.yellow.trips`
  13:WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 1
  14:AND
  15:    trip_distance > 0
  16:    AND fare_amount >= 2.5
  17:    AND pickup_longitude > -78
  18:    AND pickup_longitude < -70
  19:    AND dropoff_longitude > -78
  20:    AN

Make the validation dataset be 1/10 the size of the training dataset.

In [8]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_valid_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 10000)) = 2
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Executing query with job ID: 19bb6b94-d386-4bca-b90a-4b98bfac377d
Query executing: 0.33s


ERROR:
 404 Not found: Dataset qwiklabs-gcp-04-26c8364f1a75:taxifare was not found in location US

(job ID: 19bb6b94-d386-4bca-b90a-4b98bfac377d)

                         -----Query Job SQL Follows-----                         

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:CREATE OR REPLACE TABLE taxifare.feateng_valid_data AS
   2:
   3:SELECT
   4:    (tolls_amount + fare_amount) AS fare_amount,
   5:    pickup_datetime,
   6:    pickup_longitude AS pickuplon,
   7:    pickup_latitude AS pickuplat,
   8:    dropoff_longitude AS dropofflon,
   9:    dropoff_latitude AS dropofflat,
  10:    passenger_count*1.0 AS passengers,
  11:    'unused' AS key
  12:FROM `nyc-tlc.yellow.trips`
  13:WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 10000)) = 2
  14:AND
  15:    trip_distance > 0
  16:    AND fare_amount >= 2.5
  17:    AND pickup_longitude > -78
  18:    AND pickup_longitude < -70
  19:    AND dropoff_longitude > -78
  20:    AND

## Export the tables as CSV files

In [9]:
%%bash

echo "Deleting current contents of $OUTDIR"
gsutil -m -q rm -rf $OUTDIR

echo "Extracting training data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_training_data \
   $OUTDIR/taxi-train-*.csv

echo "Extracting validation data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_valid_data \
   $OUTDIR/taxi-valid-*.csv

# With `-l` option, gsutil will output additional information about each matching provider, bucket, subdirectory, or object.
gsutil ls -l $OUTDIR

Deleting current contents of gs://<BUCKET>/taxifare/data
Extracting training data to gs://<BUCKET>/taxifare/data
BigQuery error in extract operation: Invalid extract destination URI
'gs://<BUCKET>/taxifare/data/taxi-train-*.csv'. Must be a valid Google Cloud
Storage path and filename/pattern.
Extracting validation data to gs://<BUCKET>/taxifare/data
BigQuery error in extract operation: Invalid extract destination URI
'gs://<BUCKET>/taxifare/data/taxi-valid-*.csv'. Must be a valid Google Cloud
Storage path and filename/pattern.


BucketNotFoundException: 404 gs://<BUCKET> bucket does not exist.
CommandException: Encountered non-existent bucket during listing
BadRequestException: 400 Invalid bucket name: '<BUCKET>'


CalledProcessError: Command 'b'\necho "Deleting current contents of $OUTDIR"\ngsutil -m -q rm -rf $OUTDIR\n\necho "Extracting training data to $OUTDIR"\nbq --location=US extract \\\n   --destination_format CSV  \\\n   --field_delimiter "," --noprint_header \\\n   taxifare.feateng_training_data \\\n   $OUTDIR/taxi-train-*.csv\n\necho "Extracting validation data to $OUTDIR"\nbq --location=US extract \\\n   --destination_format CSV  \\\n   --field_delimiter "," --noprint_header \\\n   taxifare.feateng_valid_data \\\n   $OUTDIR/taxi-valid-*.csv\n\n# With `-l` option, gsutil will output additional information about each matching provider, bucket, subdirectory, or object.\ngsutil ls -l $OUTDIR\n'' returned non-zero exit status 1.

In [ ]:
# The cat command outputs the contents of one or more URLs to stdout.
!gsutil cat gs://$BUCKET/taxifare/data/taxi-train-000000000000.csv | head -2

Copyright 2021 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.